# Sending Events to S3

This notebook is pretty simple, just enough code to send events as discrete JSON files to S3 based on user interactions for Amazon Personalize.

## Setup

In [1]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
import uuid

Be sure to update the Campaign and Dataset ARNs below.

In [2]:
# Setup and Config
# Recommendations from Event data
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Campaign ARN
Campaign_ARN = "arn:aws:personalize:us-east-1:990615222287:campaign/DEMO-campaign"

# Dataset Group Arn:
datasetGroupArn = "arn:aws:personalize:us-east-1:990615222287:dataset-group/tripadvisor-dataset-group"

# Establish a connection to Personalize's Event Streaming
personalize_events = boto3.client(service_name='personalize-events')


# S3 Configuration
bucket = "personalizedemozixiqi"

## Faking Events

This code will rely on 3 users that we happen to know exist in the MovieLens dataset:

1. 196
2. 13
3. 198

They were chosen at random, also we are going to now select 3 movies at random for the user to interact with:

1. 225
2. 203
3. 242

Each of these users will interact with each movie, and each interaction will be written to S3 as a distinct JSON file.

In [3]:
users = ["196", "103", "1098"]
items = ["2025", "2003", "2402"]

In [4]:
def write_interaction_to_JSON(user, item):
    # An event needs a user, session, time, type, and a properties dictionary.
    time_of_interaction = int(time.time())
    # This is set for simplification
    session_id = user
    user_id = user
    event_type = "watched"
    properties = {
        'itemId': str(item)
    }
    properties = json.dumps(properties)
    
    event_headers = {
        'userId': user,
        'sessionId': session_id
    }
    
    event_content = {
        'sentAt': time_of_interaction,
        'eventType': event_type,
        'properties': properties
    }
    event_list = [event_content]
    json_struct = [event_headers, event_content]
    json_filename = str(time_of_interaction) + ".json"
    
    with open(json_filename, 'w') as outfile:
        json.dump(json_struct, outfile)
    
    return json_filename

def send_interactions_to_s3():
    for user in users:
        for item in items:
            filename = write_interaction_to_JSON(user, item)
            boto3.Session().resource('s3').Bucket(bucket).Object(filename).upload_file(filename)
            print(filename)
            # Sleep needed to make sure everyone gets a nice new timestamp
            time.sleep(2)
            

In [5]:
send_interactions_to_s3()

1575493241.json
1575493243.json
1575493245.json
1575493248.json
1575493250.json
1575493252.json
1575493254.json
1575493256.json
1575493259.json


### Simulate User Actions

In [ ]:
print("hello")

In [6]:
import time
max_time = time.time() + 60*60
while time.time() < max_time:
    send_interactions_to_s3()

1575493299.json
1575493301.json
1575493304.json
1575493306.json
1575493308.json
1575493310.json
1575493312.json
1575493315.json
1575493317.json
1575493319.json
1575493321.json
1575493323.json
1575493326.json
1575493328.json
1575493330.json
1575493332.json
1575493334.json
1575493337.json
1575493339.json
1575493341.json
1575493343.json
1575493345.json
1575493347.json
1575493350.json
1575493352.json
1575493354.json
1575493356.json
1575493358.json
1575493360.json
1575493363.json
1575493365.json
1575493367.json
1575493369.json
1575493371.json
1575493373.json
1575493375.json
1575493378.json
1575493380.json
1575493382.json
1575493384.json
1575493386.json
1575493388.json
1575493391.json
1575493393.json
1575493395.json
1575493397.json
1575493399.json
1575493401.json
1575493404.json
1575493406.json
1575493408.json
1575493410.json
1575493412.json
1575493414.json
1575493417.json
1575493419.json
1575493421.json
1575493423.json
1575493425.json
1575493428.json
1575493430.json
1575493432.json
15754934

In [21]:
# The code below is just to explore what to put in our Lambda:
personalize_events = boto3.client('personalize-events')
with open("1575489008.json", 'r') as input_file:
    content = json.loads(input_file.read())
    # assume tracking ID was provided via an environment variable and replace
    trackingId = "b43a6137-0aed-4fed-87b8-bfcd6a423c92"
    print(content[0]['userId'])
    
    personalize_events.put_events(
        trackingId = trackingId,
        userId = content[0]['userId'],
        sessionId = content[0]['sessionId'],
        eventList = [content[1]]
    )

196
